##pip install

In [ ]:
pip install openai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.2 MB/s eta 0:00:00


In [ ]:
pip install openai==0.28

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.35.13
    Uninstalling openai-1.35.13:
      Successfully uninstalled openai-1.35.13


In [ ]:
!pip install langchain


In [ ]:
pip install PyPDF2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.9 MB/s eta 0:00:00


In [ ]:
pip install pandas

In [ ]:
pip install -U langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.2 MB/s eta 0:00:00


##실습

In [ ]:
import openai
import csv
import json
import pandas as pd
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import CharacterTextSplitter

# Set your OpenAI API key here
openai.api_key = 'key 입력'

In [ ]:
import pandas as pd

# CSV 파일을 로드하는 함수 (한글 인코딩 지정)
def load_csv(file_path):
    try:
        # 먼저 euc-kr로 시도
        df = pd.read_csv(file_path, encoding='euc-kr',header=None)
    except UnicodeDecodeError:
        try:
            # 실패하면 latin1으로 시도
            df = pd.read_csv(file_path, encoding='latin1',header=None)
        except UnicodeDecodeError:
            # 그래도 실패하면 utf-8로 시도
            df = pd.read_csv(file_path, encoding='utf-8,',header=None)
    return df

# CSV 파일 경로
file_path = '/content/선박코드 조회.CSV'

# CSV 파일 로드
df = load_csv(file_path)

# CSV 파일의 일부를 출력
print(df.head())


    0     1      2                        3
0  순번  코드ID    코드명                    코드영문명
1  1     11    여객선           PASSENGER SHIP
2  2     12    화객선     CARGO PASSENGER SHIP
3  3     13  국제카페리  INTERNATIONAL CAR FERRY
4  4     14   크루즈선                  CRUISER


In [ ]:

# 텍스트를 기반으로 질문 생성
def generate_questions(context, num_questions=5):
    print("Context:\n", context)
    prompt = f"""
Context information is below. You are only aware of this context and nothing else.
Don't use information out of context.
    ---------------------

    {context}

    ---------------------
You are responsible for generating {num_questions} questions throughout the document, but you must not use any information from outside the document.
You are the person who generates questions based on the content of the context.
QUESTIONS must be written in Korean.

    """
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "Generate question using only the content of the context."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7,
        max_tokens=500,
    )
    questions = response.choices[0].message.content
    return questions

In [ ]:
# 수정코드
# 텍스트를 기반으로 질문 생성
def generate_questions(context, num_questions=5):
    print("Context:\n", context)
    prompt = f"""
Context information is below. You are only aware of this context and nothing else.
Don't use information out of context.
    ---------------------

    {context}

    ---------------------
You are responsible for generating {num_questions} questions throughout the document, but you must not use any information from outside the document.
You are the person who generates questions based on the content of the context.

The purpose of question(s) is to test the understanding of the students on the context information provided.
You must also provide the answer to each question. The answer should be based on the context information provided only.
QUESTIONS must be written in Korean.

    """
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "Generate question using only the content of the context."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7,
        max_tokens=500,
    )
    questions = response.choices[0].message.content
    return questions

In [ ]:

# 질문에 대한 답변 생성
def generate_answers(questions, context):
    answers = []
    for question in questions:
        prompt = f"""
Context information is below. You are only aware of this context and nothing else.
Don't use information out of context.
    ---------------------

    {context}

    ---------------------
You're an expert on ports, logistics, and ships, so when the question comes up, you're going to give an educated, expert response to the relevant regulations.
Don't use information out of context.

        Question:
        {question}

Answers must be 300 characters or less
Please condense your answer to the maximum character length.
Your answers must be written in Korean.
Answers must be in complete sentences.



 #Format:
    {{
    "QUESTION": "",
    "ANSWER": ""
    }}

        """
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "Provide answers to questions. Don't use information out of context."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
            max_tokens=300,
        )
        answer = response.choices[0].message.content
        answers.append(answer)
    return answers

In [ ]:
# 수정 코드
# 질문에 대한 답변 생성
def generate_answers(questions, context):
    answers = []
    for question in questions:
        prompt = f"""
Context information is below. You are only aware of this context and nothing else.
Don't use information out of context.
    ---------------------

    {context}

    ---------------------
You're an expert on ports, logistics, and ships, so when the question comes up, you're going to give an educated, expert response to the relevant regulations.
Don't use information out of context.

        Question:
        {question}

The answer should be based on the context information provided only.
Answers must be 150 characters or less
Please condense your answer to the maximum character length.
Your answers must be written in Korean.
Answers must be in complete sentences.



 #Format:
    {{
    "QUESTION": "",
    "ANSWER": ""
    }}

        """
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "Provide answers to questions."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
            max_tokens=300,
        )
        answer = response.choices[0].message.content
        answers.append(answer)
    return answers

In [ ]:
# CSV 파일에서 질문과 답변 쌍 생성
def generate_qa_from_csv(csv_path, num_questions=5):
    # CSV 파일 로드
    df = load_csv(csv_path)
    context = " ".join(df.astype(str).values.flatten())

    # 질문 생성
    questions = generate_questions(context, num_questions=num_questions).split("\n")

    # 답변 생성
    answers = generate_answers(questions, context)

    # Q&A 쌍 생성
    qa_pairs = [{"QUESTION": q, "ANSWER": a} for q, a in zip(questions, answers)]

    return qa_pairs

In [ ]:
# CSV 파일 경로
csv_path = '/content/선박코드 조회.CSV'
qa_pairs = generate_qa_from_csv(csv_path, num_questions=5)

Context:
 순번 코드ID 코드명 코드영문명 1  11 여객선 PASSENGER SHIP 2  12 화객선 CARGO PASSENGER SHIP 3  13 국제카페리 INTERNATIONAL CAR FERRY 4  14 크루즈선 CRUISER 5  21 산물선(벌크선) BULK SHIP 6  22 양곡운반선 CARRYING GRAIN SHIP 7  23 원목운반선 CARRYING WOOD SHIP 8  24 광석운반선 CARRYING ORE SHIP 9  25 석탄운반선 CARRYING COAL SHIP 10  26 시멘트운반선 CARRYING CEMENT SHIP 11  27 자동차운반선 CARRYING AUTOMOBILE SHIP 12  28 핫코일운반선 CARRYING HOT COIL SHIP 13  29 철강재 운반선 CARRYING STEEL PRODUCTS SHIP 14  30 코일전용선 ONLY COIL SHIP 15  31 모래운반선 CARRYING SAND SHIP 16  32 냉동.냉장선 REFRIGERATOR SHIP 17  33 폐기물 운반선 CARRYING WASTE SHIP 18  39 일반화물선 GENERAL CARGO SHIP 19  41 풀컨테이너선 FULL CONTAINER SHIP 20  42 세미(혼재)컨테이너선 SEMI CONTAINER SHIP 21  51 원유운반선 CARRYING CRUDE OIL SHIP 22  52 석유제품 운반선 CARRYING PETROLEUM PRODUCTS SHIP 23  53 케미칼 운반선 CARRYING CHEMICAL SHIP 24  54 케미칼가스 운반선 CARRYING CHEMICAL GAS SHIP 25  55 LPG 운반선 CARRYING LPG SHIP 26  56 LNG 운반선 CARRYING LNG SHIP 27  59 기타 유조선 THE OTHERS OIL SHIP 28  61 견인용예선 TUGBOAT FOR TOWING 29  62 이.접안용 예선 TUGBOAT F

In [ ]:
# 답변 출력
for qa in qa_pairs:
    print(qa['ANSWER'])

{
    "QUESTION": "1. 각 코드에 대해 설명하십시오.",
    "ANSWER": "11은 여객선을 나타내며, 21은 산물선(벌크선)을 의미합니다. 31은 모래운반선을 나타내며, 41은 풀컨테이너선을 의미합니다. 51은 원유운반선을 나타내며, 61은 견인용예선을 의미합니다."
}
{
    "QUESTION": "2. 어떤 유형의 선박이 석유제품을 운반합니까?",
    "ANSWER": "석유제품을 운반하는 선박 유형은 'CARRYING PETROLEUM PRODUCTS SHIP'입니다."
}
{
"QUESTION": "3. 어떤 유형의 선박이 산물을 운반합니까?",
"ANSWER": "산물을 운반하는 선박 유형은 벌크선입니다."
}
{
"QUESTION": "4. 코드 88은 어떤 유형의 선박을 나타내나요?",
"ANSWER": "코드 88은 무선기지부선을 나타냅니다."
}
{
    "QUESTION": "5. 코드 41은 어떤 유형의 선박을 나타내나요?",
    "ANSWER": "코드 41은 풀컨테이너선을 나타냅니다."
}
